In [0]:
from pyspark.sql.functions import col

In [0]:
bronze_transactions = spark.read.table("workspace.default.bronze_transactions")
bronze_users = spark.read.table("workspace.default.bronze_users")

In [0]:
display(
  bronze_users 
)

In [0]:
display(
  bronze_transactions .limit(10)
)

In [0]:
%sql
-- SQL Server
SELECT * 
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS
WHERE CONSTRAINT_TYPE = 'PRIMARY KEY'
  AND TABLE_NAME = 'bronze_transactions ';


In [0]:
%sql
SELECT * 
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS
WHERE CONSTRAINT_TYPE = 'PRIMARY KEY'
  AND TABLE_NAME = 'bronze_user';


In [0]:
users_clean1 = bronze_users.dropDuplicates(["id"])
display(users_clean1)

In [0]:
users_clean1.write.format("delta").mode("overwrite").saveAsTable("workspace.default.silver_users")

In [0]:
%sql
select * from silver_users

In [0]:
users_clean3 = bronze_transactions.dropDuplicates(["id"])
display(users_clean3)

In [0]:
from pyspark.sql.functions import to_date

df = bronze_transactions.withColumn(
    "date_clean",
    to_date("date")
)
display(df)

In [0]:
df = df.drop("date")
display(df)

In [0]:
%sql
SELECT
  merchant_city,
  count(*) AS merchant_city_count
FROM
  bronze_transactions
GROUP BY
  merchant_city,
  merchant_state

In [0]:
null_count = bronze_transactions.filter(
    col("merchant_state").isNull()
).count()
print(null_count)

In [0]:
df_filled1 = df.fillna(
    {"merchant_state": "N/A"}
)
display(df_filled1)

In [0]:
df_filled2 = df_filled1.fillna(
    {"zip": 0}
)
display(df_filled2)

In [0]:
df_filled2.write.format("delta").mode("overwrite").saveAsTable("workspace.default.silver_transactions")

In [0]:
%sql
select * from silver_transactions